In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import warnings
import optuna

LOAD_DATA_PATH = 'df_el.csv'
WEATHER_DATA_PATH = 'df_weather_hourly.csv'
WEATHER_TIMESTAMP_COL = 'Timestamp'
ALL_WEATHER_AND_TIME_PROVIDED_FEATURES = [
    'T', 'P0', 'P', 'U', 'Ff', 'ff10', 'VV', 'Td',
    'cloud_condition_score', 'cloud_base_meters', 'wind_x', 'wind_y',
    'wind_is_variable', 'wind_is_calm', 'weather_severity',
    'T_lag1h', 'T_lag24h', 'Td_lag1h', 'Td_lag24h', 'U_lag1h', 'U_lag24h',
    'hour', 'dayofweek', 'month', 'dayofyear', 'weekofyear',
    'is_weekend', 'is_holiday', 'FT',
    'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos', 'month_sin', 'month_cos'
]
MODEL_INPUT_WEATHER_TIME_FEATURES = ALL_WEATHER_AND_TIME_PROVIDED_FEATURES
BUILDING_LOAD_COLS = ['ICT', 'U06, U06A, U05B', 'U05, U04, U04B, GEO', 'LIB', 'MEK', 'SOC', 'S01', 'OBS', 'TEG', 'D04']
LOAD_BUILDING_ID_COL_MELTED = 'building_id'
LOAD_VALUE_COL_MELTED = 'load'
TARGET_BUILDING_ID = 'ICT' 
if TARGET_BUILDING_ID not in BUILDING_LOAD_COLS:
    raise ValueError(f"TARGET_BUILDING_ID '{TARGET_BUILDING_ID}' not found.")
TRAIN_BUILDING_IDS = [bid for bid in BUILDING_LOAD_COLS if bid != TARGET_BUILDING_ID]
B7_KNOWN_MONTHS = 2
LAG_FEATURES_LOAD = [1, 2, 24, 168]
ROLLING_WINDOW_LOAD = [3, 6, 12, 24, 24*3, 24*7] # Daily and Weekly rolling features (mean and std) 

N_TRIALS_GENERAL = 50 
N_TRIALS_FINETUNE = 30
EARLY_STOPPING_ROUNDS_OPTUNA = 30
FINETUNE_VALID_WEEKS = 2

def mape(y_true, y_pred):
    y_true_arr, y_pred_arr = np.array(y_true), np.array(y_pred)
    mask = np.abs(y_true_arr) > 1e-9 
    if np.sum(mask) == 0: return np.nan
    y_true_masked = y_true_arr[mask]
    y_pred_masked = y_pred_arr[mask]
    if len(y_true_masked) == 0: return np.nan
    mape = np.mean(np.abs((y_true_masked - y_pred_masked) / y_true_masked)) * 100
    if np.sum(~mask) > 0:
        print(f"    (MAPE calc: {np.sum(~mask)} zero/near-zero actuals excluded)")
    return mape

def load_and_prepare_data(load_path, weather_path, weather_ts_col, building_load_cols,
                          load_bid_col_melted, load_val_col_melted):
    try:
        df_el = pd.read_csv(load_path)
        df_weather = pd.read_csv(weather_path)
    except FileNotFoundError as e:
        print(f"Error: {e}. Please check your file paths.")
        raise
    df_weather[weather_ts_col] = pd.to_datetime(df_weather[weather_ts_col])
    df_weather = df_weather.sort_values(by=weather_ts_col)
    df_el_timestamped = df_el.copy()
    if weather_ts_col not in df_el_timestamped.columns:
        num_load_rows, num_weather_rows = len(df_el_timestamped), len(df_weather)
        if num_load_rows > num_weather_rows:
            df_el_timestamped = df_el_timestamped.iloc[:num_weather_rows]
        df_el_timestamped[weather_ts_col] = df_weather[weather_ts_col].values[:len(df_el_timestamped)]
    df_el_timestamped[weather_ts_col] = pd.to_datetime(df_el_timestamped[weather_ts_col])
    df_load_long = df_el_timestamped.melt(
        id_vars=[weather_ts_col], value_vars=building_load_cols,
        var_name=load_bid_col_melted, value_name=load_val_col_melted
    )
    df_merged = pd.merge(df_load_long, df_weather, on=weather_ts_col, how='left')
    df_merged = df_merged.sort_values(by=[load_bid_col_melted, weather_ts_col])
    return df_merged

def create_load_specific_features(df, timestamp_col, group_col, target_col, lags, windows):
    df = df.sort_values(by=[group_col, timestamp_col]) 
    print(f"  Generating lag features: {lags}")
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby(group_col)[target_col].shift(lag)
    
    if windows: 
        print(f"  Generating rolling window features for windows: {windows}")
        for window in windows:
            shifted_target = df.groupby(group_col)[target_col].shift(1) # Shift to prevent data leakage
            df[f'{target_col}_roll_mean_{window}'] = shifted_target.groupby(df[group_col]).rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
            df[f'{target_col}_roll_std_{window}'] = shifted_target.groupby(df[group_col]).rolling(window=window, min_periods=1).std().reset_index(level=0, drop=True)
    else:
        print("  No rolling windows specified.")
    df = df.replace([np.inf, -np.inf], np.nan)
    return df

def objective_general_model_with_tscv(trial, full_train_val_data_for_general_model, model_features_list):
    params = {
        'objective': 'regression_l1', 'metric': 'mae', 'verbose': -1, 'n_jobs': -1,
        'seed': 42, 'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators_general', 800, 2500, step=100),
        'learning_rate': trial.suggest_float('learning_rate_general', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves_general', 20, 150),
        'feature_fraction': trial.suggest_float('feature_fraction_general', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction_general', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq_general', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples_general', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha_general', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda_general', 1e-3, 10.0, log=True),
    }
    X_all_general = full_train_val_data_for_general_model[model_features_list]
    y_all_general = full_train_val_data_for_general_model[LOAD_VALUE_COL_MELTED]
    tscv = TimeSeriesSplit(n_splits=5)
    fold_maes = []
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_all_general)):
        X_train_fold, X_val_fold = X_all_general.iloc[train_idx], X_all_general.iloc[val_idx]
        y_train_fold, y_val_fold = y_all_general.iloc[train_idx], y_all_general.iloc[val_idx]
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)], eval_metric='mae',
                  callbacks=[lgb.early_stopping(EARLY_STOPPING_ROUNDS_OPTUNA, verbose=False)])
        preds_val_fold = model.predict(X_val_fold)
        mae_val_fold = mean_absolute_error(y_val_fold, preds_val_fold)
        fold_maes.append(mae_val_fold)
    if not fold_maes: return np.inf
    return np.mean(fold_maes)

def objective_finetune_target_with_validation(trial, X_target_ft_train, y_target_ft_train, X_target_ft_valid, y_target_ft_valid, init_model_for_ft):
    params_ft = {
        'objective': 'regression_l1', 'metric': 'mae', 'verbose': -1, 'n_jobs': -1,
        'seed': 123, 'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('ft_n_estimators', 30, 300, step=20),
        'learning_rate': trial.suggest_float('ft_learning_rate', 0.0005, 0.05, log=True),
        'num_leaves': trial.suggest_int('ft_num_leaves', 5, 40),
        'feature_fraction': trial.suggest_float('ft_feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('ft_bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('ft_bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('ft_min_child_samples', 2, 20),
        'reg_alpha': trial.suggest_float('ft_reg_alpha', 1e-2, 5.0, log=True),
        'reg_lambda': trial.suggest_float('ft_reg_lambda', 1e-2, 5.0, log=True),
    }
    model_ft_trial = lgb.LGBMRegressor(**params_ft)
    model_ft_trial.fit(X_target_ft_train, y_target_ft_train, 
                       eval_set=[(X_target_ft_valid, y_target_ft_valid)],
                       eval_metric='mae',
                       callbacks=[lgb.early_stopping(10, verbose=False)], 
                       init_model=init_model_for_ft) 
    preds_ft_valid = model_ft_trial.predict(X_target_ft_valid)
    mae_ft_valid = mean_absolute_error(y_target_ft_valid, preds_ft_valid)
    return mae_ft_valid

# --- Main Script ---
def main():
    print("Starting Load Forecasting")
    print(f"Target Building ID for this run: {TARGET_BUILDING_ID}")
    print(f"Training Building IDs: {TRAIN_BUILDING_IDS}")
    print(f"Lag features: {LAG_FEATURES_LOAD}")
    print(f"Rolling window features: {ROLLING_WINDOW_LOAD}")


    # 1. Load and Prepare Data
    print("\n--- 1. Loading and Preparing Data ---")
    data_df_raw = load_and_prepare_data(
        LOAD_DATA_PATH, WEATHER_DATA_PATH, WEATHER_TIMESTAMP_COL,
        BUILDING_LOAD_COLS, LOAD_BUILDING_ID_COL_MELTED, LOAD_VALUE_COL_MELTED
    )
    target_data_check = data_df_raw[data_df_raw[LOAD_BUILDING_ID_COL_MELTED] == TARGET_BUILDING_ID]
    if target_data_check.empty: raise ValueError(f"No data for {TARGET_BUILDING_ID}")
    target_first_ts = target_data_check[WEATHER_TIMESTAMP_COL].min()
    target_known_load_end_date = target_first_ts + pd.DateOffset(months=B7_KNOWN_MONTHS) - pd.Timedelta(hours=1)
    target_forecast_start_date = target_known_load_end_date + pd.Timedelta(hours=1)
    target_forecast_end_date = min(target_forecast_start_date + pd.DateOffset(months=10) - pd.Timedelta(hours=1), data_df_raw[WEATHER_TIMESTAMP_COL].max())
    print(f"Target Building '{TARGET_BUILDING_ID}': Known: {target_first_ts} to {target_known_load_end_date}, Forecast: {target_forecast_start_date} to {target_forecast_end_date}")

    # 2. Feature Engineering
    print("\n--- 2. Engineering Load-Specific Features ---")
    data_df_fe = create_load_specific_features(
        data_df_raw.copy(), WEATHER_TIMESTAMP_COL, LOAD_BUILDING_ID_COL_MELTED, LOAD_VALUE_COL_MELTED,
        LAG_FEATURES_LOAD, ROLLING_WINDOW_LOAD # ROLLING_WINDOW_LOAD is now active
    )
    generated_load_features = [col for col in data_df_fe.columns if LOAD_VALUE_COL_MELTED + "_lag_" in col or LOAD_VALUE_COL_MELTED + "_roll_" in col]
    MODEL_FEATURE_COLUMNS = MODEL_INPUT_WEATHER_TIME_FEATURES + generated_load_features
    MODEL_FEATURE_COLUMNS = sorted(list(set(col for col in MODEL_FEATURE_COLUMNS if col in data_df_fe.columns)))
    print(f"Total features for model (after adding rolling windows): {len(MODEL_FEATURE_COLUMNS)}")

    # 3. Prepare Full Data for General Model TSCV
    print("\n--- 3. Preparing Full Data for General Model TSCV ---")
    full_general_model_data_unclean = data_df_fe[data_df_fe[LOAD_BUILDING_ID_COL_MELTED].isin(TRAIN_BUILDING_IDS)].copy()
    full_general_model_data_clean = full_general_model_data_unclean.dropna(subset=[LOAD_VALUE_COL_MELTED] + MODEL_FEATURE_COLUMNS).copy()
    if full_general_model_data_clean.empty:
        raise ValueError("No data for general model after initial NaN drop for TSCV.")
    
    print(f"\n--- Optuna: Tuning General Model with TimeSeriesSplit ({N_TRIALS_GENERAL} trials) ---")
    study_general = optuna.create_study(direction='minimize', study_name=f'general_model_tscv_{TARGET_BUILDING_ID}_iter1')
    study_general.optimize(
        lambda trial: objective_general_model_with_tscv(trial, full_general_model_data_clean, MODEL_FEATURE_COLUMNS),
        n_trials=N_TRIALS_GENERAL, show_progress_bar=True
    )
    best_general_params_optuna = study_general.best_params
    print("Optuna Best hyperparameters for general model (from TSCV): ", best_general_params_optuna)
    print(f"Optuna Best average MAE for general model (from TSCV): {study_general.best_value:.4f}")
    
    print("\n--- 4. Training Final General LightGBM Model ---")
    general_model_train_val_df = data_df_fe[data_df_fe[LOAD_BUILDING_ID_COL_MELTED].isin(TRAIN_BUILDING_IDS)].copy()
    all_train_timestamps_final = sorted(general_model_train_val_df[WEATHER_TIMESTAMP_COL].unique())
    split_idx_final = int(len(all_train_timestamps_final) * 0.8)
    validation_start_time_final = all_train_timestamps_final[split_idx_final]
    train_df_general_final = general_model_train_val_df[general_model_train_val_df[WEATHER_TIMESTAMP_COL] < validation_start_time_final]
    valid_df_general_final = general_model_train_val_df[general_model_train_val_df[WEATHER_TIMESTAMP_COL] >= validation_start_time_final]
    train_df_general_final_clean = train_df_general_final.dropna(subset=[LOAD_VALUE_COL_MELTED] + MODEL_FEATURE_COLUMNS)
    valid_df_general_final_clean = valid_df_general_final.dropna(subset=[LOAD_VALUE_COL_MELTED] + MODEL_FEATURE_COLUMNS)
    if train_df_general_final_clean.empty or valid_df_general_final_clean.empty:
        raise ValueError("Final general model train or validation data empty after NaN drop.")
    X_train_general_final = train_df_general_final_clean[MODEL_FEATURE_COLUMNS]
    y_train_general_final = train_df_general_final_clean[LOAD_VALUE_COL_MELTED]
    X_valid_general_final = valid_df_general_final_clean[MODEL_FEATURE_COLUMNS]
    y_valid_general_final = valid_df_general_final_clean[LOAD_VALUE_COL_MELTED]

    final_general_model_params = {
        'objective': 'regression_l1', 'metric': 'mae', 'verbose': -1, 'n_jobs': -1,
        'seed': 42, 'boosting_type': 'gbdt',
    }
    final_general_model_params.update({key.replace('_general', ''): value for key, value in best_general_params_optuna.items()})
    
    general_lgbm_model = lgb.LGBMRegressor(**final_general_model_params)
    general_lgbm_model.fit(X_train_general_final, y_train_general_final,
                           eval_set=[(X_valid_general_final, y_valid_general_final)],
                           eval_metric='mae',
                           callbacks=[lgb.early_stopping(EARLY_STOPPING_ROUNDS_OPTUNA + 20, verbose=True)])
    
    mae_general_valid_final = mean_absolute_error(y_valid_general_final, general_lgbm_model.predict(X_valid_general_final))
    mape_general_valid_final = mape(y_valid_general_final, general_lgbm_model.predict(X_valid_general_final))
    print(f"Final General Model Validation MAE (on 20% holdout): {mae_general_valid_final:.4f}, MAPE: {mape_general_valid_final:.2f}%")

    target_finetune_full_known_data = data_df_fe[
        (data_df_fe[LOAD_BUILDING_ID_COL_MELTED] == TARGET_BUILDING_ID) &
        (data_df_fe[WEATHER_TIMESTAMP_COL] >= target_first_ts) &
        (data_df_fe[WEATHER_TIMESTAMP_COL] <= target_known_load_end_date)
    ].copy()
    target_finetune_full_known_data_clean = target_finetune_full_known_data.dropna(subset=[LOAD_VALUE_COL_MELTED] + MODEL_FEATURE_COLUMNS)
    if target_finetune_full_known_data_clean.empty:
        raise ValueError(f"No data for fine-tuning {TARGET_BUILDING_ID}.")

    target_ft_timestamps = sorted(target_finetune_full_known_data_clean[WEATHER_TIMESTAMP_COL].unique())
    X_target_ft_train_hpo, y_target_ft_train_hpo, X_target_ft_valid_hpo, y_target_ft_valid_hpo = (None, None, None, None)

    if len(target_ft_timestamps) < FINETUNE_VALID_WEEKS * 7 * 24 * 1.5 : 
         print(f"Warning: target known data too short for a {FINETUNE_VALID_WEEKS}-week validation split during HPO. Using all for objective.")
         X_target_ft_train_hpo = target_finetune_full_known_data_clean[MODEL_FEATURE_COLUMNS]
         y_target_ft_train_hpo = target_finetune_full_known_data_clean[LOAD_VALUE_COL_MELTED]
         X_target_ft_valid_hpo = X_target_ft_train_hpo 
         y_target_ft_valid_hpo = y_target_ft_train_hpo
    else:
        target_ft_valid_start_offset = pd.DateOffset(weeks=FINETUNE_VALID_WEEKS)
        potential_validation_start_time = target_finetune_full_known_data_clean[WEATHER_TIMESTAMP_COL].max() - target_ft_valid_start_offset + pd.Timedelta(hours=1)
        target_ft_validation_start_time = max(potential_validation_start_time, target_finetune_full_known_data_clean[WEATHER_TIMESTAMP_COL].min() + pd.Timedelta(days=1)) 
        target_ft_train_hpo_df = target_finetune_full_known_data_clean[target_finetune_full_known_data_clean[WEATHER_TIMESTAMP_COL] < target_ft_validation_start_time]
        target_ft_valid_hpo_df = target_finetune_full_known_data_clean[target_finetune_full_known_data_clean[WEATHER_TIMESTAMP_COL] >= target_ft_validation_start_time]
        if target_ft_train_hpo_df.empty or target_ft_valid_hpo_df.empty:
            print("Warning: target fine-tune HPO train or validation split is empty. Using all target known data for HPO objective.")
            X_target_ft_train_hpo = target_finetune_full_known_data_clean[MODEL_FEATURE_COLUMNS]
            y_target_ft_train_hpo = target_finetune_full_known_data_clean[LOAD_VALUE_COL_MELTED]
            X_target_ft_valid_hpo = X_target_ft_train_hpo
            y_target_ft_valid_hpo = y_target_ft_train_hpo
        else:
            X_target_ft_train_hpo = target_ft_train_hpo_df[MODEL_FEATURE_COLUMNS]
            y_target_ft_train_hpo = target_ft_train_hpo_df[LOAD_VALUE_COL_MELTED]
            X_target_ft_valid_hpo = target_ft_valid_hpo_df[MODEL_FEATURE_COLUMNS]
            y_target_ft_valid_hpo = target_ft_valid_hpo_df[LOAD_VALUE_COL_MELTED]
    print(f"target HPO Fine-tune train size: {len(X_target_ft_train_hpo if X_target_ft_train_hpo is not None else [])}, HPO Fine-tune valid size: {len(X_target_ft_valid_hpo if X_target_ft_valid_hpo is not None else [])}")

    print(f"\n--- Optuna: Tuning target Fine-Tuning Parameters ({N_TRIALS_FINETUNE} trials) ---")
    study_finetune = optuna.create_study(direction='minimize', study_name=f'target_finetune_validated_{TARGET_BUILDING_ID}_iter1')
    study_finetune.optimize(
        lambda trial: objective_finetune_target_with_validation(
            trial, X_target_ft_train_hpo, y_target_ft_train_hpo, 
            X_target_ft_valid_hpo, y_target_ft_valid_hpo, general_lgbm_model),
        n_trials=N_TRIALS_FINETUNE, show_progress_bar=True
    )
    best_finetune_params_optuna = study_finetune.best_params
    print("Optuna Best hyperparameters for target fine-tuning: ", best_finetune_params_optuna)
    print(f"Optuna Best MAE on target fine-tune validation: {study_finetune.best_value:.4f}")

    print(f"\n--- 5. Fine-tuning Final Model for {TARGET_BUILDING_ID} ---")
    X_target_finetune_final = target_finetune_full_known_data_clean[MODEL_FEATURE_COLUMNS]
    y_target_finetune_final = target_finetune_full_known_data_clean[LOAD_VALUE_COL_MELTED]
    final_finetune_model_params = {
        'objective': 'regression_l1', 'metric': 'mae', 'verbose': -1, 'n_jobs': -1,
        'seed': 123, 'boosting_type': 'gbdt',
    }
    final_finetune_model_params.update({key.replace('ft_', ''): value for key, value in best_finetune_params_optuna.items()})
    target_finetuned_model = lgb.LGBMRegressor(**final_finetune_model_params)
    target_finetuned_model.fit(X_target_finetune_final, y_target_finetune_final, init_model=general_lgbm_model)
    mae_target_finetune_on_own = mean_absolute_error(y_target_finetune_final, target_finetuned_model.predict(X_target_finetune_final))
    mape_target_finetune_on_own = mape(y_target_finetune_final, target_finetuned_model.predict(X_target_finetune_final))
    print(f"target MAE on its own full 2-month data (after final fine-tuning): {mae_target_finetune_on_own:.4f}, MAPE: {mape_target_finetune_on_own:.2f}%")

    print(f"\n--- 6. Recursive Forecasting for {TARGET_BUILDING_ID} (using Fine-tuned Model) ---")
    target_forecast_df_full = data_df_fe[data_df_fe[LOAD_BUILDING_ID_COL_MELTED] == TARGET_BUILDING_ID].copy()
    target_forecast_df_full = target_forecast_df_full.sort_values(by=WEATHER_TIMESTAMP_COL).set_index(WEATHER_TIMESTAMP_COL)
    target_actuals_for_forecast_period = data_df_raw[
        (data_df_raw[LOAD_BUILDING_ID_COL_MELTED] == TARGET_BUILDING_ID) &
        (data_df_raw[WEATHER_TIMESTAMP_COL] >= target_forecast_start_date) &
        (data_df_raw[WEATHER_TIMESTAMP_COL] <= target_forecast_end_date)
    ][LOAD_VALUE_COL_MELTED].values
    target_predictions_recursive = []
    forecast_timestamps_target = pd.date_range(start=target_forecast_start_date, end=target_forecast_end_date, freq='H')
    target_working_forecast_df = target_forecast_df_full.copy()
    for current_time_step in forecast_timestamps_target:
        current_features_series = target_working_forecast_df.loc[current_time_step].copy()
        X_pred_step_df = pd.DataFrame([current_features_series[MODEL_FEATURE_COLUMNS]], columns=MODEL_FEATURE_COLUMNS)
        if X_pred_step_df.isnull().values.any(): X_pred_step_df = X_pred_step_df.fillna(0) 
        predicted_load_step = target_finetuned_model.predict(X_pred_step_df)[0]
        target_predictions_recursive.append(predicted_load_step)
        target_working_forecast_df.loc[current_time_step, LOAD_VALUE_COL_MELTED] = predicted_load_step
        next_ts_for_lag_update = current_time_step + pd.Timedelta(hours=1)
        if next_ts_for_lag_update <= target_forecast_end_date and f'{LOAD_VALUE_COL_MELTED}_lag_1' in target_working_forecast_df.columns:
            if next_ts_for_lag_update in target_working_forecast_df.index : # Ensure the index exists
                 target_working_forecast_df.loc[next_ts_for_lag_update, f'{LOAD_VALUE_COL_MELTED}_lag_1'] = predicted_load_step
    target_predictions_recursive = np.array(target_predictions_recursive)
    
    if len(target_actuals_for_forecast_period) == len(target_predictions_recursive) and len(target_actuals_for_forecast_period) > 0 :
        target_forecast_mae = mean_absolute_error(target_actuals_for_forecast_period, target_predictions_recursive)
        target_forecast_mape = mape(target_actuals_for_forecast_period, target_predictions_recursive)
        print(f"\nMAE on {TARGET_BUILDING_ID}'s forecast (original scale, after fine-tuning): {target_forecast_mae:.4f}, MAPE: {target_forecast_mape:.2f}%")
    else:
        min_len = min(len(target_actuals_for_forecast_period), len(target_predictions_recursive))
        if min_len > 0 :
            target_forecast_mae_partial = mean_absolute_error(target_actuals_for_forecast_period[:min_len], target_predictions_recursive[:min_len])
            target_forecast_mape_partial = mape(target_actuals_for_forecast_period[:min_len], target_predictions_recursive[:min_len])
            print(f"\nMAE on {TARGET_BUILDING_ID}'s forecast (partial {min_len} steps, original scale, after fine-tuning): {target_forecast_mae_partial:.4f}, MAPE: {target_forecast_mape_partial:.2f}%")
        else:
            print(f"\nCould not calculate MAE/MAPE for {TARGET_BUILDING_ID} forecast period (actuals length: {len(target_actuals_for_forecast_period)}, preds length: {len(target_predictions_recursive)}).")

    # print("\n--- 7. Saving Submission File ---")
    # if len(forecast_timestamps_target) == len(target_predictions_recursive):
    #     submission_df_lgbm = pd.DataFrame({'timestamp': forecast_timestamps_target, 'building_id': TARGET_BUILDING_ID,'predicted_load': target_predictions_recursive})
    #     submission_df_lgbm.to_csv(f'submission_{TARGET_BUILDING_ID}.csv', index=False)
    #     print(f"Submission file 'submission_{TARGET_BUILDING_ID}.csv' saved.")
    # else:
    #     print("Warning: Submission not saved due to length mismatch.")


if __name__ == '__main__':

    main()

Starting Load Forecasting
Target Building ID for this run: ICT
Training Building IDs: ['U06, U06A, U05B', 'U05, U04, U04B, GEO', 'LIB', 'MEK', 'SOC', 'S01', 'OBS', 'TEG', 'D04']
Lag features: [1, 2, 24, 168]
Rolling window features: [3, 6, 12, 24, 72, 168]

--- 1. Loading and Preparing Data ---
Target Building 'ICT': Known: 2023-01-01 00:00:00 to 2023-02-28 23:00:00, Forecast: 2023-03-01 00:00:00 to 2023-12-31 23:00:00

--- 2. Engineering Load-Specific Features ---
  Generating lag features: [1, 2, 24, 168]
  Generating rolling window features for windows: [3, 6, 12, 24, 72, 168]


[I 2025-05-15 14:38:43,226] A new study created in memory with name: general_model_tscv_ICT_iter1


Total features for model (after adding rolling windows): 51

--- 3. Preparing Full Data for General Model TSCV ---

--- Optuna: Tuning General Model with TimeSeriesSplit (50 trials) ---


Best trial: 0. Best value: 4.52969:   2%|▏         | 1/50 [00:15<12:23, 15.17s/it]

[I 2025-05-15 14:38:58,391] Trial 0 finished with value: 4.529694603755091 and parameters: {'n_estimators_general': 900, 'learning_rate_general': 0.02200546858537131, 'num_leaves_general': 90, 'feature_fraction_general': 0.46309214150227396, 'bagging_fraction_general': 0.9291252566592622, 'bagging_freq_general': 6, 'min_child_samples_general': 36, 'reg_alpha_general': 0.006692775310255614, 'reg_lambda_general': 0.1431708780045997}. Best is trial 0 with value: 4.529694603755091.


...

[I 2025-05-15 14:53:54,421] Trial 28 finished with value: 7.184285513147337 and parameters: {'ft_n_estimators': 190, 'ft_learning_rate': 0.02166041197258928, 'ft_num_leaves': 36, 'ft_feature_fraction': 0.5510420454880184, 'ft_bagging_fraction': 0.6402699032643084, 'ft_bagging_freq': 2, 'ft_min_child_samples': 15, 'ft_reg_alpha': 0.023009476136524685, 'ft_reg_lambda': 0.01602720021183695}. Best is trial 21 with value: 6.954216019948878.


Best trial: 21. Best value: 6.95422: 100%|██████████| 30/30 [00:14<00:00,  2.03it/s]


[I 2025-05-15 14:53:55,435] Trial 29 finished with value: 7.342918791035895 and parameters: {'ft_n_estimators': 230, 'ft_learning_rate': 0.0032244276311355693, 'ft_num_leaves': 33, 'ft_feature_fraction': 0.5914409122074458, 'ft_bagging_fraction': 0.7119797487643198, 'ft_bagging_freq': 1, 'ft_min_child_samples': 17, 'ft_reg_alpha': 1.2555582693071174, 'ft_reg_lambda': 0.3578272827285108}. Best is trial 21 with value: 6.954216019948878.
Optuna Best hyperparameters for target fine-tuning:  {'ft_n_estimators': 190, 'ft_learning_rate': 0.046736202405970456, 'ft_num_leaves': 37, 'ft_feature_fraction': 0.5380392719224733, 'ft_bagging_fraction': 0.5289315283303215, 'ft_bagging_freq': 3, 'ft_min_child_samples': 18, 'ft_reg_alpha': 0.03682399990297231, 'ft_reg_lambda': 0.010552807905311667}
Optuna Best MAE on target fine-tune validation: 6.9542

--- 5. Fine-tuning Final Model for ICT ---
target MAE on its own full 2-month data (after final fine-tuning): 3.6166, MAPE: 2.23%

--- 6. Recursive Fore

C:\Users\ashooshtari\AppData\Local\Temp\ipykernel_15984\4283948840.py:302: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  forecast_timestamps_target = pd.date_range(start=target_forecast_start_date, end=target_forecast_end_date, freq='H')



MAE on ICT's forecast (original scale, after fine-tuning): 15.0316, MAPE: 8.29%

--- 7. Saving Submission File ---
Submission file 'submission_lgbm_iter1_rolling_ICT.csv' saved.

LGBM Process Iteration 1 (Rolling Windows) Finished.
